# Election map visualization

Bayu Wilson

In `integration_and_model_training.ipynb` I used data from the MIT Election lab, the Decennial Census, and the American Community Survey to make an integrated dataset. I also performed feature engineering to extract useful predictors for flipped counties. Finally, I trained an XGBoost classifier to predict whether counties are likely to flip or not.

We will be visualizing geospatial data using a Python library called [Folium](https://python-visualization.github.io/folium/latest/).

Loading in useful libraries

In [163]:
import json
import folium
from branca.element import MacroElement
from jinja2 import Template
import branca.colormap as cm
import pandas as pd
from IPython.display import IFrame

Useful functions for colormaps

In [46]:
class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)
        
step = cm.StepColormap(
    ["red", "white","blue"], vmin=-1, vmax=1, 
    caption="Flipped counties from previous election. -1 = flip to R, 0 = no flip, 1 = flip to D")

### Taking a look at the geojson file

GeoJSON is a format for encoding a variety of geographic data structures. In this case, it contains shape information about counties in the USA.

`STATEFP` is for state identifier. For example, 06 is California

`COUNTYFP` is for county identifier. 

Combine them together to get `GEOID`. For example, 06075 is San Francisco county, California.

In [164]:
geojson_file = "data/counties.geojson"
with open(geojson_file) as f:
    geojson_data = json.load(f)
geojson_data['features'][0]['properties'] #key on "feature.properties.GEOID"

{'STATEFP': '06',
 'COUNTYFP': '075',
 'COUNTYNS': '00277302',
 'AFFGEOID': '0500000US06075',
 'GEOID': '06075',
 'NAME': 'San Francisco',
 'LSAD': '06',
 'ALAND': 121485107,
 'AWATER': 479107241}

loading in flipped county data

In [166]:
df_total_before = pd.read_csv("data/df_total_2016-2020.csv")
df_total_future = pd.read_csv("data/df_total_2022.csv")
df_total_before["flipped"] = df_total_before["flipped"].map({0:0,1:1,2:-1}) #-1 (flip to R classification) mapped to 2
df_total_future["flipped"] = df_total_future["flipped"].map({0:0,1:1,2:-1}) #XGBoost likes classes to be >=0

### Map visualization functions

In [170]:
year_list = df_total_before['year'].unique()

def get_marker(coords,text,color):
    marker = folium.Marker(coords, popup=folium.Popup(text, parse_html=True, max_width=100),
            icon=folium.Icon(color=color,icon='star', prefix='fa'))
    return marker

def make_map():
    m = folium.Map(tiles=None,location=[39.0997, -94.5786], zoom_start=4)
    base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
    folium.TileLayer(tiles='cartodb positron',min_zoom=4).add_to(base_map)
    
    base_map.add_to(m)
    
    df_list = [df_total_before[df_total_before["year"]==year_list[0]],
              df_total_before[df_total_before["year"]==year_list[1]],
              df_total_future[df_total_future["year"]==2022]]
    year_range_str_list = ["2012-2016","2016-2020","2020-2024 prediction"]
    for i in range(3):
        df_i = df_list[i].copy()
        df_i = df_i.reset_index()

        year_range_str = year_range_str_list[i]
        choropleth = folium.Choropleth(
            geo_data=geojson_file,
            name=year_range_str,
            data=df_i,
            columns=["county_fips", "flipped"],
            key_on="feature.properties.GEOID",
            fill_color="RdBu",
            colormap=step,
            fill_opacity=0.5,
            line_opacity=0.1,
            bins=3,
            show=i==2, 
            overlay=False,
        )

        county_labels = folium.GeoJson(geojson_file,
        name='Counties',
        tooltip=folium.GeoJsonTooltip(
            fields=['NAME'],  # Column from GeoJSON to display
            aliases=['County Name:'],  # Label for the tooltip
            localize=True
        ),overlay=True,control=False,
        style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent', 'weight': 0}  # Hide borders
        )
        
        for child in choropleth._children:
            if child.startswith("color_map"):
                del choropleth._children[child]

        if i ==2:
            text_flip2D = "top 3 most populous counties predicted to flip to D"
            color_flip2D = "blue"
            coords_flip2D = [[40.9849, -72.6151],[42.7169, -82.8210],[33.1795, -96.4930]]

            text_flip2R = "top 3 most populous counties predicted to flip to R"
            color_flip2R = "red"
            coords_flip2R = [[37.5091, -120.9876],[42.9931, -71.0498],[42.0522, -80.1875]]

            for j in range(3):
                marker_D = get_marker(coords_flip2D[j],text_flip2D,color_flip2D)
                marker_D.add_to(choropleth)
                marker_R = get_marker(coords_flip2R[j],text_flip2R,color_flip2R)
                marker_R.add_to(choropleth)
            
        m.add_child(step)
        county_labels.add_to(choropleth)
        m.add_child(choropleth)
        bc = BindColormap(choropleth, step)
        m.add_child(bc)
    

    folium.LayerControl(collapsed=False, position='topright', overlay=False).add_to(m)

    m.save('map.html')

    # Display the map using an IFrame with custom width and height
    return IFrame('map.html', width=800, height=600)

In [172]:
make_map()